## Streamgraph for Analysis of China's Steel Production

### Importation of packages

In [7]:
library(devtools)
library(streamgraph)
library(dplyr)
library(babynames)
library(reshape2)
library(shiny)
library(scales)
library(ggplot2)
library(tidyverse)
library(htmlwidgets)
library(RColorBrewer)
library(tidyr)
library(htmltools)
library(magrittr)
library(xts)
library(IRdisplay)

Loading required package: usethis

Loading required package: htmlwidgets

Loading required package: htmltools


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v forcats   1.0.0     v stringr   1.5.0
v lubridate 1.9.2     v tibble    3.2.1
v purrr     1.0.1     v tidyr     1.3.0
v readr     2.1.4     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x readr::col_factor() masks scales::col_factor()
x purrr::discard()    masks scales::discard()
x dplyr::filter()     masks stats::filter()
x dplyr::lag()        masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: 'magrittr'


The following object is masked from 'package:purrr':

    set_names


### Preparation of data frame for streamgraph


In [8]:
#Loading and displaying the steel production dataframe:
load('r_from_pd_steel_df.RData')
head(r_from_pd_steel_df)

,France,Germany,Italy,Spain,United Kingdom,Poland,Turkey,Russia,Canada,Mexico,United States,Brazil,China,India,Japan,South Korea,Rest of World
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1917,2231651,16587360,1296200,0,9639000,0,0,0,1587400,0,45166184,0,0,160000,770000,0,4999952
1918,1807931,14979505,992523,0,9399000,0,0,0,1722200,0,44461210,0,0,180000,810000,0,2656889
1919,2186260,7847356,731823,250000,7855000,0,0,0,942500,0,34562995,0,0,189900,810000,20000,2203076
1920,3050396,9277882,773761,260000,9002000,980000,0,160000,1127700,30000,41896719,0,0,158700,843000,30000,4569288
1921,3102170,9996538,700433,306300,3658000,850000,0,180000,680200,40000,19630315,0,0,185600,864000,50000,4477609
1922,4534492,11714302,980719,314300,5888000,1000000,0,310000,488800,50000,35330515,0,0,152900,935000,10000,6227496


In [10]:
#Adding a column for the year(although there are duplicates, the melt function will utilize the year column that is named 'Year):')
r_from_pd_steel_df$Year <- 1917:2022
r_from_pd_steel_df


,France,Germany,Italy,Spain,United Kingdom,Poland,Turkey,Russia,Canada,Mexico,United States,Brazil,China,India,Japan,South Korea,Rest of World,Year
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1917,2231651,16587360,1296200,0,9639000,0,0,0,1587400,0,45166184,0,0,160000,770000,0,4999952,1917
1918,1807931,14979505,992523,0,9399000,0,0,0,1722200,0,44461210,0,0,180000,810000,0,2656889,1918
1919,2186260,7847356,731823,250000,7855000,0,0,0,942500,0,34562995,0,0,189900,810000,20000,2203076,1919
1920,3050396,9277882,773761,260000,9002000,980000,0,160000,1127700,30000,41896719,0,0,158700,843000,30000,4569288,1920
1921,3102170,9996538,700433,306300,3658000,850000,0,180000,680200,40000,19630315,0,0,185600,864000,50000,4477609,1921
1922,4534492,11714302,980719,314300,5888000,1000000,0,310000,488800,50000,35330515,0,0,152900,935000,10000,6227496,1922
1923,5109517,6305250,1141761,475700,8465000,1130000,0,600000,898700,60000,44467466,0,0,219000,986000,0,7084371,1923
1924,6900310,9835255,1358853,540000,8162000,680000,0,1000000,671500,40000,37599059,0,0,340800,1099700,0,9140414,1924
1925,7446463,12194501,1784673,580100,7503900,780000,0,1870000,768600,70000,45091520,0,0,456300,1336391,0,9517404,1925


In [11]:
#Reshape your dataframe into a long format where each row represents a combination of country, year, and value.Ensure that the reshaped dataframe has three columns: country, year, and value. The country column should contain the names of the countries, the year column should contain the corresponding years, and the value column should contain the values for each country at each year.
melted_df <- melt(r_from_pd_steel_df, id.vars ="Year", variable.name = "Country", value.name = "Steel")
melted_df


Year,Country,Steel
<int>,<fct>,<dbl>
1917,France,2231651
1918,France,1807931
1919,France,2186260
1920,France,3050396
1921,France,3102170
1922,France,4534492
1923,France,5109517
1924,France,6900310
1925,France,7446463


In [12]:
#Placing the 'China' Country category at the top of the dataframe:

# Identify the current name used for China in the data
current_name <- levels(melted_df$Country)[which(levels(melted_df$Country) != "China")]

# Change the name to "China"
melted_df$Country <- factor(melted_df$Country, levels = c("China", levels(melted_df$Country)[levels(melted_df$Country) != "China"]))

# Verify the change
levels(melted_df$Country)

[1] "China"          "France"         "Germany"        "Italy"         
 [5] "Spain"          "United Kingdom" "Poland"         "Turkey"        
 [9] "Russia"         "Canada"         "Mexico"         "United States" 
[13] "Brazil"         "India"          "Japan"          "South Korea"   
[17] "Rest of World"

### Plotting streamgraph and displaying

In [22]:

#Creating a streamgraph using the melted_df:
steel_streamgraph_1917_2022<-streamgraph(
    melted_df, 
    key='Country', #Name of categorical variable
    value='Steel', #Name of the value column (the measured element; y-axis)
    date='Year', #Name of the x-axis
    width="675px", #Width of the stream graph itself
    interpolate="linear", #Style of the stramgraph
    offset='silhouette') %>% #Form of the stream graph
    sg_fill_manual(c('#140c6e','#FFCC00','#CC0000','#bdbaba','#34332d','#b65f25d5','#2e7b2a','#666699','#b1afaf','#fffefe','#006c65','#990000','#660066','#19197b','#333300','#5f3c76','#006666','#2A8386')) %>% #Setting up the colors of the graph manually; the colors are set in alphabetical order based on the country names.
    sg_axis_y(0) %>% #Setting the measurement of the y-axis
    sg_legend(show=TRUE, label = "Country: ") %>% #Setting the legend based on the categorical values of 'Country'
    sg_add_marker(x=as.Date("1932-01-01"), label="Great Depression",space=2,size=10)%>%
    sg_add_marker(x=as.Date("1938-01-01"),y=221664847, label="1937-1938 Recession",space=2,size=10) %>%
    sg_add_marker(x=as.Date("1945-01-01"),y=421664847, label="1945 Recession",space=2,size=10)%>%
    sg_add_marker(x=as.Date("1958-01-01"),y=601664847, label="1958 Recession",space=2,size=10)%>%
    sg_add_marker(x=as.Date("1971-01-01"),y=1401664847, label="1969-1970 Recession",space=2,size=10)%>%
    sg_add_marker(x=as.Date("1975-01-01"),y=1501664847, label="1973-1975 Recession",space=2,size=10)%>%
    sg_add_marker(x=as.Date("1982-01-01"),y=1601664847, label="1981-1982 Recession",space=2,size=10)%>%
    sg_add_marker(x=as.Date("2009-01-01"),y=1901664847, label="2008 World Economic Crisis",space=2,size=10,anchor="end")%>% #Adding a marker to the graph based on economic crises
    sg_annotate(label="Collapse of USSR", x=as.Date("1988-01-01"), y=851664847, color='white',size=8) #Adding an annotation of an important historical event to the graph.



In [433]:
#Save the widget: Use the saveWidget function to save the stream graph as an HTML widget
saveWidget(steel_streamgraph_1917_2022, "steel_streamgraph_1917_2022.html", selfcontained = TRUE)


Warning message in widget_html(name, package, id = x$id, style = css(width = validateCssUnit(sizeInfo$width), :
"streamgraph_html returned an object of class `list` instead of a `shiny.tag`."
Warning message:
"`bindFillRole()` only works on htmltools::tag() objects (e.g., div(), p(), etc.), not objects of type 'list'. "


In [399]:
#Extracting the first element of the list:
steel_streamgraph_tag <- steel_streamgraph_1917_2022[[1]]
steel_streamgraph_tag


key,value,date
<fct>,<dbl>,<chr>
China,0,1917-01-01
France,2231651,1917-01-01
Germany,16587360,1917-01-01
Italy,1296200,1917-01-01
Spain,0,1917-01-01
United Kingdom,9639000,1917-01-01
Poland,0,1917-01-01
Turkey,0,1917-01-01
Russia,0,1917-01-01


In [21]:
#Display the streamgraph in the browser:
browseURL("steel_streamgraph_1917_2022.html")